In [1]:
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.applications.resnet import preprocess_input
import os
import numpy as np


def get_imlist(path):
    return [os.path.join(path, f) for f in os.listdir(path) if f.endswith(u'.jpg')]


def create_image_dict(img_list):
    input_shape = (224, 224, 3)
    tensor = {}
    for path in img_list:
        img = image.load_img(path, target_size=(input_shape[0], input_shape[1]))
        img = image.img_to_array(img)
        img = preprocess_input(img)
        img_key = path.strip('holidays_small/')
        tensor[img_key] = img
    # tensor = np.array(tensor)
    return tensor

Using TensorFlow backend.


In [2]:
img_dict = create_image_dict(get_imlist('holidays_small'))
img_dict.keys()

dict_keys(['138708.jpg', '134001.jpg', '138604.jpg', '107500.jpg', '124300.jpg', '123100.jpg', '122301.jpg', '115600.jpg', '130600.jpg', '101502.jpg', '141201.jpg', '146003.jpg', '128601.jpg', '106903.jpg', '139603.jpg', '101200.jpg', '118202.jpg', '112602.jpg', '139505.jpg', '121101.jpg', '113403.jpg', '108601.jpg', '138303.jpg', '133703.jpg', '148101.jpg', '131803.jpg', '132902.jpg', '130401.jpg', '123402.jpg', '104100.jpg', '139103.jpg', '106602.jpg', '106702.jpg', '122400.jpg', '106201.jpg', '106402.jpg', '139801.jpg', '117403.jpg', '142302.jpg', '111500.jpg', '118300.jpg', '121700.jpg', '137400.jpg', '101400.jpg', '132401.jpg', '106300.jpg', '146501.jpg', '139205.jpg', '134401.jpg', '126702.jpg', '132102.jpg', '138905.jpg', '105102.jpg', '112000.jpg', '120101.jpg', '106801.jpg', '138302.jpg', '134601.jpg', '113402.jpg', '133901.jpg', '139301.jpg', '112500.jpg', '148301.jpg', '119400.jpg', '104002.jpg', '147402.jpg', '109500.jpg', '137801.jpg', '141800.jpg', '132402.jpg', '109802.j

In [3]:
# create dataset
queries = []
positives = []
negatives = []

triplets_file = open("triplets.dat", "r")
for line in triplets_file.readlines():
    split = line.split(" ")[:3]

    queries.append(img_dict[split[0]])
    positives.append(img_dict[split[1]])
    negatives.append(img_dict[split[2]])

queries = np.array(queries)
positives = np.array(positives)
negatives = np.array(negatives)

In [4]:
#create model
from keras.applications import ResNet50

embedding_size = 512

input_shape = (224, 224, 3)

resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=input_shape)

for layer in resnet.layers:
    layer.trainable = False
    #print(layer, layer.trainable)

resnet.layers.pop(0)

In [5]:
#set layers untrainable
from keras.models import Model
from keras.layers import Input,  Dense, Dropout
from keras.optimizers import Adam
from keras.utils import plot_model
from triplet_loss import L2NormLayer

input_q = Input(shape=(224, 224, 3))
input_p = Input(shape=(224, 224, 3))
input_n = Input(shape=(224, 224, 3))

resnet_q = resnet(input_q)
resnet_p = resnet(input_p)
resnet_n = resnet(input_n)

resnet_output = resnet.output_shape[1]



"""l2normalization = L2NormLayer()
embedding = Dense(embedding_size, input_shape=(resnet_output,), activation='relu', name="embedding1")
dropout = Dropout(0.1)
"""

In [6]:
embedding_q = dropout(embedding(l2normalization(resnet_q)))
embedding_p = dropout(embedding(l2normalization(resnet_p)))
embedding_n = dropout(embedding(l2normalization(resnet_n)))

resnet_qpn = Model([input_q, input_p, input_n], [embedding_q, embedding_p, embedding_n])

In [7]:
plot_model(resnet_qpn, to_file='base_network.png', show_shapes=True, show_layer_names=True)
resnet_qpn.summary()

result = resnet_qpn.predict([queries[:1], positives[:1], negatives[:1]])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 2048)         23587712    input_2[0][0]                    
                                                                 input_3[0][0]              

In [8]:
all_data_len = len(img_dict.keys())
n_train = 333

fake_true_pred = np.zeros((n_train,embedding_size*3))
fake_true_pred_val = np.zeros((all_data_len-n_train,embedding_size*3))

queries_train = queries[:n_train]
positives_train = positives[:n_train]
negatives_train = negatives[:n_train]

queries_test = queries[n_train:]
positives_test = positives[n_train:]
negatives_test = negatives[n_train:]

In [9]:
from triplet_loss import TripletLossLayer

batch_size = 16
epochs = 12

# train session
opt = Adam(lr=0.0001)  # choose optimiser. RMS is good too!

loss_layer = TripletLossLayer(alpha=.1,name='triplet_loss_layer')(resnet_qpn.output)
resnet_qpn = Model(inputs = resnet_qpn.input, outputs=loss_layer)
resnet_qpn.compile(optimizer=opt)

/home/fede/Documenti/netvlad/venv/lib/python3.6/site-packages/keras/engine/training_utils.py:819: UserWarning: Output triplet_loss_layer missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to triplet_loss_layer.
  'be expecting any data to be passed to {0}.'.format(name))


In [10]:
H = resnet_qpn.fit(
            x=[queries_train, positives_train, negatives_train],
            y=None,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=([queries_test, positives_test, negatives_test], None),
            verbose=1,
            )

import matplotlib.pyplot as plt

resnet_qpn.save_weights("model.h5")
print("Saved model to disk")

        
plt.figure(figsize=(8,8))
plt.plot(H.history['loss'], label='training loss')
plt.plot(H.history['val_loss'], label='validation loss')
plt.legend()
plt.title('Train/validation loss')
plt.show()

Train on 333 samples, validate on 1158 samples
Epoch 1/12
333/333 [==============================] - 25s 75ms/step - loss: 0.1481 - val_loss: 0.1115
Epoch 2/12
333/333 [==============================] - 16s 49ms/step - loss: 0.0531 - val_loss: 0.0952
Epoch 3/12
333/333 [==============================] - 16s 49ms/step - loss: 0.0347 - val_loss: 0.0875
Epoch 4/12
333/333 [==============================] - 16s 48ms/step - loss: 0.0099 - val_loss: 0.0830
Epoch 5/12
333/333 [==============================] - 16s 49ms/step - loss: 0.0034 - val_loss: 0.0806
Epoch 6/12
333/333 [==============================] - 16s 49ms/step - loss: 0.0038 - val_loss: 0.0789
Epoch 7/12
333/333 [==============================] - 16s 48ms/step - loss: 0.0025 - val_loss: 0.0771
Epoch 8/12
333/333 [==============================] - 16s 48ms/step - loss: 0.0022 - val_loss: 0.0764
Epoch 9/12
333/333 [==============================] - 16s 48ms/step - loss: 0.0014 - val_loss: 0.0741
Epoch 10/12
333/333 [==============

<Figure size 800x800 with 1 Axes>

In [11]:
# pop triplet loss layer
# resnet_qpn.layers.pop()

result = resnet_qpn.predict([queries[:1], positives[:1], negatives[:1]])


In [12]:
# reload model from disk
resnet_qpn = Model([input_q, input_p, input_n], [embedding_q, embedding_p, embedding_n])

resnet_qpn.load_weights('model.h5')

In [13]:
# test model 

#this function create a perfect ranking :)

from sklearn.neighbors import NearestNeighbors

input_shape = (224,224,3)

def get_imlist_(path="holidays_small"):
    imnames = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(u'.jpg')]
    imnames = [path.strip('holidays_small/') for path in imnames]
    imnames = [path.strip('.jpg') for path in imnames]
    return imnames

def images_to_tensor(imnames):
  images_array = []

  # open all images
  for name in imnames:
    img_path = 'holidays_small/'+ name +'.jpg'
    img = image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    images_array.append(img)
  images_array = np.array(images_array)
  print(images_array.shape)
  # images_array = preprocess_input(images_array)
  return images_array

imnames = get_imlist_()

query_imids = [i for i, name in enumerate(imnames) if name[-2:].split('.')[0] == "00"]

# check that everything is fine - expected output: "tot images = 1491, query images = 500"
print('tot images = %d, query images = %d' % (len(imnames), len(query_imids)))

tot images = 1491, query images = 500


In [14]:
# img_tensor = images_to_tensor(imnames)
img_tensor = [img_dict[key] for key in img_dict]
img_tensor = np.array(img_tensor)

In [15]:
all_feats, _, _ = resnet_qpn.predict([img_tensor, np.zeros(img_tensor.shape), np.zeros(img_tensor.shape)]) 

In [16]:
query_feats = all_feats[query_imids]

#SOLUTION
nbrs = NearestNeighbors(n_neighbors=1491, metric='cosine').fit(all_feats)
distances, indices = nbrs.kneighbors(query_feats)

In [17]:
def make_perfect_holidays_result(imnames, q_ids):
  perfect_idx =[]
  for qimno in q_ids:
      qname = imnames[qimno]
      positive_results = set([i for i, name in enumerate(imnames) if name != qname and name[:4] == qname[:4]])
      ok=[qimno]+[i for i in  positive_results]
      others = [i for i in range(1491) if i not in positive_results and i != qimno]
      perfect_idx.append(ok+others)
  return np.array(perfect_idx)


def mAP(q_ids, idx):
  aps = []
  for qimno, qres in zip(q_ids, idx):
      qname = imnames[qimno]
      # collect the positive results in the dataset
      # the positives have the same prefix as the query image
      positive_results = set([i for i, name in enumerate(imnames)
                              if name != qname and name[:4] == qname[:4]])
      #
      # ranks of positives. We skip the result #0, assumed to be the query image
      ranks = [i for i, res in enumerate(qres[1:]) if res in positive_results]
      #
      # accumulate trapezoids with this basis
      recall_step = 1.0 / len(positive_results)
      ap = 0
      for ntp,rank in enumerate(ranks):
          # ntp = nb of true positives so far
          # rank = nb of retrieved items so far
          # y-size on left side of trapezoid:
          precision_0 = ntp/float(rank) if rank > 0 else 1.0
          # y-size on right side of trapezoid:
          precision_1 = (ntp + 1) / float(rank + 1)
          ap += (precision_1 + precision_0) * recall_step / 2.0
      #print('query %s, AP = %.3f' % (qname, ap))
      aps.append(ap)
  return np.mean(aps)


print('mean AP = %.3f'%mAP(query_imids, indices))
perfect_result=make_perfect_holidays_result(imnames, query_imids)
print('Perfect mean AP = %.3f'%mAP(query_imids,perfect_result))

mean AP = 0.765
Perfect mean AP = 1.000
